In [3]:
import os
import cv2
import rasterio
import numpy as np
from PIL import Image
import geopandas as gpd
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import box
from rasterio.windows import Window

In [240]:
def getJPGData(path):
    '''
    get all jpg maindata froom RGB path
    '''
    jpgDataList = []

    #loop into day folder
    for mainFolder in os.listdir(path):
        mainFolderPath = path + '/' + mainFolder

        #loop into label folder
        for labelFolder in os.listdir(mainFolderPath):
            labelFolderPath = mainFolderPath + '/' + labelFolder

            # loop into every file
            for dataFile in os.listdir(labelFolderPath):

                # check if the file is jpg
                if(dataFile[-4:] == ".jpg"):
                    dataFilePath = labelFolderPath + '/' + dataFile

                    # add jpg file
                    jpgDataList.append(dataFilePath)
    return jpgDataList

def normalize_image(image):
    """
    Normalizes image to range 0-255.
    """
    norm_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    return norm_image.astype(np.uint8)

def apply_gamma_correction(image, gamma):
    """
    Applies gamma correction.
    """
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype(np.uint8)
    return cv2.LUT(image, table)

def apply_clahe(image, clip_limit, tile_grid_size):
    """
    Applies Contrast Limited Adaptive Histogram Equalization (CLAHE).
    """
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)  # Convert to LAB color space
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)  # Apply CLAHE to the L-channel
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # Convert back to BGR

def adjust_saturation(image, scale):
    """
    Adjusts image saturation by scaling the S-channel in HSV.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[..., 1] = np.clip(hsv[..., 1] * scale, 0, 255)  # Scale saturation
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def apply_sharpening(image, sigma, strength):
    """
    Applies Unsharp Masking for sharpening.
    """
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    sharpened = cv2.addWeighted(image, 1 + strength, blurred, -strength, 0)
    return np.clip(sharpened, 0, 255).astype(np.uint8)

def enhance_jpg_image(input_path, output_path):
    """
    Enhances a JPG image by applying:
    - Normalization
    - Gamma Correction (1.2)
    - CLAHE Contrast Adjustment (clip_limit=0, tile_grid_size=(24,24))
    - Saturation Adjustment (Scale 1.2)
    - Sharpening (sigma=1.5, strength=0.9)

    Parameters:
    - input_path: Path to the input JPG file.
    - output_path: Path to save the enhanced JPG.
    """
    # Load image
    image = cv2.imread(input_path)

    if image is None:
        print(f"❌ Error: Unable to load {input_path}")
        return

    # Apply enhancements
    image = normalize_image(image)
    image = apply_gamma_correction(image, gamma=0.9)
    image = apply_clahe(image, clip_limit=0.7, tile_grid_size=(12, 12))
    image = adjust_saturation(image, scale=1.15)
    image = apply_sharpening(image, sigma=1, strength=0.5)


    # Save enhanced image
    cv2.imwrite(output_path, image, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    print(f"✅ Enhanced image saved: {output_path}")

def createNewFolder(targetFilePath, foldername):
    ''' 
    create new folder for enhanced files
    '''
    prevFolderNameList = []
    
    # loop in to every target path
    for file in targetFilePath:

        # get only previouse folder path, preparing for create new folder
        prevFolderName = file.split('/')[:-1]
        prevFolderName = '/'.join(prevFolderName)

        # get rid off same files in same folder
        if(prevFolderName not in prevFolderNameList):
            prevFolderNameList.append(prevFolderName)
            
            # name new folder
            enhanceFolderName = prevFolderName + '/' + foldername

            # check if the folder already exist
            if(not(os.path.exists(enhanceFolderName))):
                # make new folder
                os.mkdir(enhanceFolderName)
                print("created folder: ", enhanceFolderName)
            else:
                print("folder already exist")
    
def loopEnhanceJPG(targetPathList):
    ''' 
    loop target file to enhance 
    '''
    count = 1

    # loop each jpg file
    for targetPath in targetPathList:

        # create output path in Enhanced folder
        splittedPath = targetPath.split('/')
        prevFolderPath = splittedPath[:-1]
        outFilePath = '/'.join(prevFolderPath) + "/" + "Enhanced" + "/" + splittedPath[-1][:-4] + "_enhanced.jpg" 

        # check if file not already exist
        if(not(os.path.exists(outFilePath))):
            enhance_jpg_image(targetPath, outFilePath)
        else:
            print("file already exist")
        print(f"{count} / {len(targetPathList)} : {count*100/len(targetPathList):.2f} %")
        count += 1

In [241]:
RGBDataPath = "D:/ice-wheat/data/dataForProcess/mainData/RGB"
jpgDataList = getJPGData(RGBDataPath)
# createNewFolder(jpgDataList, "Enhanced")
loopEnhanceJPG(jpgDataList)
# enhance_jpg_image("test1.jpg", "./test1_enhanced.jpg")
# enhance_jpg_image("test2.jpg", "./test2_enhanced.jpg")
# enhance_jpg_image("test3.jpg", "./test3_enhanced.jpg")
# enhance_jpg_image("test4.jpg", "./test4_enhanced.jpg")
# enhance_jpg_image("test5.jpg", "./test5_enhanced.jpg")

file already exist
1 / 38065 : 0.00 %
file already exist
2 / 38065 : 0.01 %
file already exist
3 / 38065 : 0.01 %
file already exist
4 / 38065 : 0.01 %
file already exist
5 / 38065 : 0.01 %
file already exist
6 / 38065 : 0.02 %
file already exist
7 / 38065 : 0.02 %
file already exist
8 / 38065 : 0.02 %
file already exist
9 / 38065 : 0.02 %
file already exist
10 / 38065 : 0.03 %
file already exist
11 / 38065 : 0.03 %
file already exist
12 / 38065 : 0.03 %
file already exist
13 / 38065 : 0.03 %
file already exist
14 / 38065 : 0.04 %
file already exist
15 / 38065 : 0.04 %
file already exist
16 / 38065 : 0.04 %
file already exist
17 / 38065 : 0.04 %
file already exist
18 / 38065 : 0.05 %
file already exist
19 / 38065 : 0.05 %
file already exist
20 / 38065 : 0.05 %
file already exist
21 / 38065 : 0.06 %
file already exist
22 / 38065 : 0.06 %
file already exist
23 / 38065 : 0.06 %
file already exist
24 / 38065 : 0.06 %
file already exist
25 / 38065 : 0.07 %
file already exist
26 / 38065 : 0.

KeyboardInterrupt: 